# Overview:

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = """airline-438510-25841398e32c.json""" 
from google.cloud import bigquery

print("Initializing BigQuery client...")
client = bigquery.Client()
print("success")

Initializing BigQuery client...
success


In [2]:
# print all project ids
projects = client.list_projects()

for project in projects:
    print(project.project_id)

airline-438510


In [17]:
# print all datasets in the project
datasets = client.list_datasets()

for dataset in datasets:
    print(dataset.dataset_id)

airlines


In [3]:
# print all tables in the dataset
tables = client.list_tables("airline-438510.airlines")

for table in tables:
    print(table.table_id)

airlines
airlines_model
average_airline_rating_by_year
average_profit_margin_by_class
avg_ticket_price_by_departure_city
customer_satisfaction_analysis
flight_duration_vs_ticket_price
market_share_by_airline
monthly_profit_trends
most_profitable_routes_by_year
profit_and_flights_by_country
profits_comparison_2022_2023
rating
revenue_breakdown_by_class_type
revenue_by_country
seasonal_passenger_trends
top5
top_destinations_by_passenger_volume
underperforming_flights


In [4]:
import pandas as pd

# print 5 rows from the table
query = """
SELECT * 
FROM `airline-438510.airlines.airlines` 
limit 1
"""

client.query(query).result().to_dataframe()

,airline_id,airline_name,airline_rating,airline_country,departure_city,arrival_city,flight_duration,number_of_passengers,ticket_price,total_revenue,costs,profit,flight_status,class_type,flight_date
0,1,Air Alpha,1.9,China,Paris,Berlin,9.53,286,1815.16,519135.76,6038.77,513096.99,On Time,Economy,2023-06-27


In [6]:
project_id = "airline-438510"
dataset_id = "airlines"
table_id = "airlines"

a_table = f"{project_id}.{dataset_id}.{table_id}"
print(a_table)

airline-438510.airlines.airlines


# Overview Page

In [11]:
# Profit from 2023 compared with 2022
query = f"""
WITH profits23 as (
SELECT airline_name, SUM(profit) AS profit_2023
FROM `{a_table}`
WHERE EXTRACT(YEAR FROM flight_date) = 2023
GROUP BY airline_name
),
profits22 as (
SELECT airline_name, SUM(profit) AS profit_2022
FROM `{a_table}`
WHERE EXTRACT(YEAR FROM flight_date) = 2022
GROUP BY airline_name
)

SELECT p23.airline_name, p23.profit_2023, p22.profit_2022, p23.profit_2023 - p22.profit_2022 as profit_diff
FROM profits23 p23
JOIN profits22 p22 ON p23.airline_name = p22.airline_name
ORDER BY profit_diff DESC
"""
q1 = client.query(query).result().to_dataframe()


AttributeError: 'DataFrame' object has no attribute 'name'

In [43]:
# Most profitable routes yearwise

query = f""" 
WITH profits as (
SELECT departure_city,
        arrival_city,
        EXTRACT(YEAR FROM flight_date) as year,
        SUM(profit) AS profit
FROM `{a_table}`
GROUP BY departure_city, arrival_city, year
),
max_profit as (
            SELECT year, MAX(profit) as max_profit
            FROM profits
            GROUP BY year
)
SELECT p.departure_city, p.arrival_city, p.year, p.profit
FROM profits p
JOIN max_profit mp ON p.year = mp.year AND p.profit = mp.max_profit
ORDER BY p.year ASC
"""

q2 = client.query(query).result().to_dataframe()
type(q2)

pandas.core.frame.DataFrame

[2024-10-21 16:35:32.001][Taipy][ERROR] Exception on /taipy-jsx/TaiPy_root_page [GET]
Traceback (most recent call last):
  File "C:\Users\Raghu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raghu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raghu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\flask_cors\extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^

Traceback (most recent call last):
  File "C:\Users\Raghu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gevent\pywsgi.py", line 1107, in handle_one_response
    self.run_application()
  File "C:\Users\Raghu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geventwebsocket\handler.py", line 87, in run_application
    return super(WebSocketHandler, self).run_application()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raghu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gevent\pywsgi.py", line 1053, in run_application
    self.result = self.application(self.environ, self.start_response)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raghu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.

: 

In [32]:
# Average Profit Margin by Class Type
# DIMENSIONS: class_type, total_revenue, costs, profit, number_of_passengers
# profit_margin = ((total_revenue - costs) / total_revenue) * 100
query = f""" 
SELECT class_type, ROUND ( avg( ( (total_revenue - costs) / total_revenue) * 100 ), 2) as average_profit_margin
FROM `{a_table}`
GROUP BY class_type
ORDER BY average_profit_margin DESC
"""

q3 = client.query(query).result().to_dataframe()
q3.columns

Index(['class_type', 'average_profit_margin'], dtype='object')

In [33]:
# Monthly Profit Trends 

query = f"""
WITH monthly_profits AS (
        SELECT airline_name, EXTRACT(MONTH FROM flight_date) as month,
                   SUM(profit) as total_monthly_profit,
                   COUNT(*) as number_of_flights
        FROM `{a_table}`
        GROUP BY airline_name, month
)
SELECT airline_name,
           ROUND(AVG(total_monthly_profit), 2) as avg_monthly_profit,
           ROUND( AVG(number_of_flights), 2)  as avg_no_of_flights
FROM monthly_profits
GROUP BY airline_name
ORDER BY avg_monthly_profit DESC
"""

q4 = client.query(query).result().to_dataframe()
q4.columns

Index(['airline_name', 'avg_monthly_profit', 'avg_no_of_flights'], dtype='object')

In [34]:
# Average Airline Rating Comparison by Year

query = f"""
WITH airline_ratings AS (
    SELECT *
    FROM (
        SELECT 
            airline_name, 
            EXTRACT(YEAR FROM flight_date) AS year,
            ROUND(AVG(airline_rating), 2) AS avg_airline_rating
        FROM `{a_table}`
        GROUP BY airline_name, year
    )
    PIVOT (
        AVG(avg_airline_rating) FOR year IN (2021, 2022, 2023)
    )
    ORDER BY _2021 DESC, _2022 DESC, _2023 DESC
)
SELECT *,
       CASE 
            WHEN _2021 > _2022 AND _2021 > _2023 THEN "Decreasing Trend"
            WHEN _2023 > _2022 AND _2023 > _2021 THEN "Increasing Trend"
            ELSE "No Trend"
       END AS trend
FROM airline_ratings
ORDER BY trend
"""
q5 = client.query(query).result().to_dataframe()
q5.columns



Index(['airline_name', '_2021', '_2022', '_2023', 'trend'], dtype='object')

In [35]:
# Market Share by Airline 
# Market share = (Total Revenue by Airline / Total Revenue) * 100

query = f"""
SELECT airline_name,
         ROUND(SUM(total_revenue) / (SELECT SUM(total_revenue) FROM `{a_table}`) * 100, 2) as market_share
FROM `{a_table}`
GROUP BY airline_name
ORDER BY market_share DESC
"""
q6 = client.query(query).result().to_dataframe()
q6.columns

Index(['airline_name', 'market_share'], dtype='object')

# Marketing Page

In [36]:
# Top destinations by passenger volume

query = f"""
SELECT arrival_city as destination, COUNT(*) as passenger_volume
from `{a_table}`
GROUP BY destination
ORDER BY passenger_volume DESC
"""

q7 = client.query(query).result().to_dataframe()
q7.columns

Index(['destination', 'passenger_volume'], dtype='object')

In [37]:
# Average ticket price by departure city

query = f"""
SELECT departure_city, ROUND(AVG(ticket_price), 2) as avg_ticket_price
FROM `{a_table}`
GROUP BY departure_city
ORDER BY avg_ticket_price DESC
"""

q8 = client.query(query).result().to_dataframe()
q8.columns

Index(['departure_city', 'avg_ticket_price'], dtype='object')

In [38]:
# Revenue breakdown by Class Type

query = f"""
SELECT class_type, 
        sum(total_revenue) / COUNT(*) as avg_revenue_per_flight, 
        sum(total_revenue) / SUM(number_of_passengers) as avg_revenue_per_passenger
FROM `{a_table}`
GROUP BY class_type
ORDER BY avg_revenue_per_flight DESC
"""

q9 = client.query(query).result().to_dataframe()
q9.columns

Index(['class_type', 'avg_revenue_per_flight', 'avg_revenue_per_passenger'], dtype='object')

In [65]:
# Seasonal Passenger trends

query = f"""
WITH seasonal_passenger_trends as (
SELECT EXTRACT(MONTH FROM flight_date) as month,
        COUNT(*) as number_of_passengers
FROM `{a_table}`
GROUP BY month
ORDER BY month
)
SELECT *,
        CASE WHEN month IN (12, 1, 2) THEN "Winter"
                WHEN month IN (3, 4, 5) THEN "Spring"
                WHEN month IN (6, 7, 8) THEN "Summer"
                ELSE "Fall"
        END as season
          FROM seasonal_passenger_trends
"""

client.query(query).result().to_dataframe()

,month,number_of_passengers,season
0,1,82,Winter
1,2,76,Winter
2,3,91,Spring
3,4,92,Spring
4,5,93,Spring
5,6,69,Summer
6,7,87,Summer
7,8,70,Summer
8,9,81,Fall
9,10,87,Fall


In [69]:
# Customer satisfaction by Analysis (co-relation between airling rating and profit)

# Pearson Co-Relation Formula: P = Cov(X, Y) / (σx * σy)
# Cov(X, Y) = Σ((X - μx) * (Y - μy)) / n
# where μx = mean of X, μy = mean of Y, n = number of observations

query = f"""
SELECT CORR(avg_airline_rating, avg_profit) AS pearson_correlation
FROM (
    SELECT 
        airline_name, 
        ROUND(AVG(airline_rating), 2) AS avg_airline_rating,
        ROUND(AVG(profit), 2) AS avg_profit
    FROM `{a_table}`
    GROUP BY airline_name
)
"""
client.query(query).result().to_dataframe()

,pearson_correlation
0,0.021346


In [70]:
df.columns

Index(['airline_id', 'airline_name', 'airline_rating', 'airline_country',
       'flight_date', 'departure_city', 'arrival_city', 'flight_duration',
       'number_of_passengers', 'ticket_price', 'total_revenue', 'costs',
       'profit', 'flight_status', 'class_type'],
      dtype='object')

In [73]:
# Flight duration impact on ticket price.

query = f"""
SELECT departure_city, arrival_city, flight_duration, AVG(ticket_price) as avg_ticket_price
FROM `{a_table}`
GROUP BY departure_city, arrival_city, flight_duration
ORDER BY flight_duration DESC, avg_ticket_price DESC
"""
client.query(query).result().to_dataframe()

,departure_city,arrival_city,flight_duration,avg_ticket_price
0,Berlin,Beijing,15.00,1992.47
1,Sao Paulo,Toronto,14.99,181.10
2,New York,Mumbai,14.97,373.56
3,Sydney,Paris,14.94,514.52
4,Beijing,Sao Paulo,14.93,1461.51
...,...,...,...,...
989,Tokyo,London,1.09,1944.48
990,New York,Beijing,1.07,1936.43
991,London,Mumbai,1.06,1158.66
992,Berlin,Mumbai,1.05,1851.30


In [74]:
# Revenue by Country
query = f"""
select distinct airline_country, sum(total_revenue) as total_revenue
from `{a_table}`
group by airline_country
order by total_revenue desc
"""
client.query(query).result().to_dataframe()

,airline_country,total_revenue
0,USA,22401880.36
1,China,19598888.99
2,Brazil,19100982.48
3,Japan,18699195.17
4,Canada,18284797.99
5,India,17726365.97
6,Australia,17412508.24
7,France,17404998.07
8,UK,17024909.28
9,Germany,16130749.15


In [81]:
# Find the total profit made by each airline country and list them in descending order. Display the airline country, number of flights, and total profit.

query = f"""
        WITH countrywise_profits as
                        (
                        SELECT airline_country, sum(profit) as total_profit
                        from {a_table}
                        GROUP BY airline_country
                        ),
            countrywise_flights as 
                        (
                        SELECT airline_country, count (distinct airline_id) as no_of_flights
                        from {a_table}
                        GROUP BY airline_country
                        )
        SELECT countrywise_profits.airline_country, countrywise_flights.no_of_flights, countrywise_profits.total_profit
        from countrywise_profits
                join countrywise_flights on countrywise_profits.airline_country = countrywise_flights.airline_country
        """

client.query(query).result().to_dataframe()

,airline_country,no_of_flights,total_profit
0,China,10,14484110.87
1,Brazil,10,13757295.61
2,Japan,10,13648276.51
3,UK,10,12516953.73
4,Australia,10,12762674.23
5,Germany,10,11827019.41
6,France,10,11635159.03
7,India,10,13059883.62
8,Canada,10,12571292.09
9,USA,10,15776070.37


In [62]:
# Find Under Performing Flights

query = """
SELECT airline_name, profit from airlines
where profit < 5000
"""

conn.sql(query).df()

,airline_name,profit
0,Air Alpha,4437.92
1,Echo Airways,-51241.58
2,Hotel Air,-507.10
3,Echo Airways,-10930.60
4,Charlie Air,-62767.20
...,...,...
162,Foxtrot Aviation,-25154.69
163,Juliet Jet,-40062.47
164,Charlie Air,-15437.55
165,Hotel Air,-37520.08


In [75]:
# Most Profitable Flight Routes

query = f"""
SELECT departure_city, arrival_city, sum(profit) as total_profit from {a_table}
group by departure_city, arrival_city
order by total_profit desc
"""

client.query(query).result().to_dataframe()

,departure_city,arrival_city,total_profit
0,Sao Paulo,Tokyo,3242650.46
1,Beijing,Sao Paulo,2769414.39
2,Paris,Berlin,2613293.97
3,London,Berlin,2525225.03
4,New York,Mumbai,2424257.99
...,...,...,...
85,Sao Paulo,New York,726989.26
86,Sydney,Sao Paulo,663513.13
87,Beijing,Tokyo,456188.02
88,Paris,Tokyo,360845.05


In [41]:
import requests
from bs4 import BeautifulSoup

# Base URL of the charts directory
base_url = "https://github.com/Avaiga/taipy/tree/develop/doc/gui/examples/charts"

# Fetch the directory listing
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all .py files in the directory
py_files = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href and href.endswith('.py'):
        # Extract the file name from the href
        py_files.append(href.split('/')[-1])

# Function to download and save the .py files
def save_files_to_txt(py_files, output_file):
    base_raw_url = "https://raw.githubusercontent.com/Avaiga/taipy/develop/doc/gui/examples/charts/"
    with open(output_file, "w", encoding='utf-8') as f_out:  # Ensure utf-8 encoding
        for file_name in py_files:
            try:
                # Fetch each file's content
                response = requests.get(base_raw_url + file_name)
                response.raise_for_status()  # Check for HTTP errors
                f_out.write(f"\n\n# {file_name}\n\n")  # Write filename as header
                f_out.write(response.text)  # Write file content
            except Exception as e:
                print(f"Failed to download {file_name}: {e}")

# Save the Python files to a text file
save_files_to_txt(py_files, "taipy_charts_examples.txt")
